In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4

In [2]:
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: fccc8e42-f40b-4a63-8828-a1530cc397fa


In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [5]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [6]:
llm = ChatOpenAI(temperature=0.5, model=llm_model)

In [7]:
graph = Neo4jGraph(
    url="neo4j://localhost:7687",
    username="neo4j",
    password="BRU0109al"
)

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

In [13]:
movie_chat = prompt | llm | StrOutputParser()

In [14]:
youtube = YouTubeSearchTool()

In [15]:
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)


In [21]:
# Define a custom tool for querying the specific database
def query_specific_database(query):
    # Implement the logic to query the specific database and return the results
    # For example:
    results = graph.run(query).data()
    return results

In [23]:
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
    name="Database Query",
    description="Use this tool to query the specific database. The input will be a query string.",
    func=query_specific_database,
    ),
]

In [24]:
agent_prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [25]:
chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])

AttributeError: 'Neo4jGraph' object has no attribute 'run'